In [1]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# ChromeDriver 的下載管理工具
# from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 下載檔案的工具
import wget

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)

# 建立儲存圖片、影片的資料夾
folderPath = 'youtube'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 放置爬取的資料
listData = []

In [45]:
# 走訪頁面
def visit():
    driver.get('https://www.youtube.com/')

# 輸入關鍵字
def search():
    # 輸入名稱
    txtInput = driver.find_element(By.CSS_SELECTOR, "input#search")
    txtInput.send_keys("烏薩奇")
    # 等待一下
    sleep(1)
    
    # 送出表單資料
    txtInput.submit()

    # 等待一下
    sleep(1)

# 篩選 (選項)
def filterFunc():
    try:
        # 等待篩選元素出現
        WebDriverWait(driver,10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "ytd-button-renderer.style-scope.ytd-search-header-renderer")
            )
        )
         #按下篩選元素，使項目浮現
        driver.find_element(
            By.CSS_SELECTOR,
            'ytd-button-renderer.style-scope.ytd-search-header-renderer'
        ).click()
        # 等待一下
        sleep(2)

        # 按下選擇的項目
        # 總共有27個所以要用s，傳回的是一個list
        driver.find_elements( 
            By.CSS_SELECTOR,"div#options a#endpoint.yt-simple-endpoint.style-scope.ytd-search-filter-renderer"
        )[9].click()

        # 等待一下
        sleep(2)
    except TimeoutException:
        print('等待逾時！')
# 滾動頁面
def scroll():
    '''
    innerHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 累計無效滾動次數
    limit => 最大無效滾動次數
    '''
    innerHeight = 0
    offset = 0
    count = 0
    limit = 3

    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # 每次移動高度
        offset = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )

        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset},
                behavior: 'smooth'
            }})
        ''')

        break

        # (重要)強制等待
        sleep(3)

        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )

        # 經過計算
        if offset == innerHeight:
            count += 1

# 分析頁面元素資訊
def parse():
    # 使用全域變數
    global listData

    # 清空存放資料的變數
    listData.clear()

    # 取得主要元素的集合
    elements = driver.find_elements(
        By.CSS_SELECTOR, 
        'ytd-video-renderer.style-scope.ytd-item-section-renderer'
    )
    # 逐一檢視元素
    for elm in elements:
        # 印出分隔文字
        print("=" * 30)
        
        # 取得圖片連結
        img = elm.find_element(
            By.CSS_SELECTOR,
            'a#thumbnail img'
        )
        imgSrc = img.get_attribute('src')
        print(imgSrc)

        # 取得超連結
        a = elm.find_element(
            By.CSS_SELECTOR, 
            'a#video-title'
        )
        aTitle = a.get_attribute('innerText')
        print(aTitle)

        # 取得 YouTube 連結
        aLink = a.get_attribute('href')
        print(aLink)

        # 取得 影音 ID (建議使用正規表達式取得youtube_id)
        strDelimiter = ''
        if 'shorts' in aLink:
            strDelimiter = '/shorts/'
        else:
            strDelimiter = 'v='
        youtube_id = aLink.split(strDelimiter)[1]
        youtube_id = youtube_id.split('&pp')[0]
        print(youtube_id)

        # 放資料到 list 中
        listData.append({
            'id': youtube_id,
            'title': aTitle,
            'link': aLink,
            'img': imgSrc
        })
# 將 list 存成 json
def saveJson():
    with open(f'{folderPath}/youtube.json', 'w', encoding='utf-8') as file:
        file.write(
            json.dumps(listData, ensure_ascii=False, indent=4))

# 關閉瀏覽器
def close():
    driver.quit()



In [ ]:
visit()
search()
filterFunc()
scroll()
parse()
saveJson()

In [47]:
close()

In [4]:
# 下載
def download():
    # 確認 yt-dlp 是否存在
    if not os.path.exists('./yt-dlp.exe'):
        print('[下載 yt-dlp]')
        wget.download(
            "https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp.exe", 
            "./yt-dlp.exe"
        )
    # 開啟 json 檔案
    with open(f"{folderPath}/youtube.json", "r", encoding='utf8') as file:
        #取得 json 字串
        strJson = file.read()

    # 將 json 轉成 list (裡面是 dict 集合)
    listResult = json.loads(strJson)

    # 下載檔案
    for index, obj in enumerate(listResult):
        if index == 3:
            break

        print(f"正在下載: {obj['link']}")

        # 定義指令
        cmd = [
            './yt-dlp.exe', 
            obj['link'], 
            '-f', 'b[ext=mp4]',
            '-o', f'{folderPath}/%(id)s.%(ext)s'
        ]

        # 執行指令，並取得回傳結果 (subprocess 物件)
        obj_sp = subprocess.run(cmd)

        # 判斷指令行是否正常 (returncode == 0 代表正常)
        if obj_sp.returncode == 0:
            print('下載成功！')
        else:
            print('下載失敗...')


download()

正在下載: https://www.youtube.com/watch?v=BLeZ9r0rJIQ&pp=ygUJ54OP6Jap5aWH
下載成功！
正在下載: https://www.youtube.com/watch?v=8oDW26PYQ3M&pp=ygUJ54OP6Jap5aWH
下載成功！
正在下載: https://www.youtube.com/watch?v=yPW7MDJU7h8&pp=ygUJ54OP6Jap5aWH
下載成功！
